In [ ]:
#default_exp criteria

# Criteria

In [ ]:
#export
import torch

In [ ]:
#hide
from nbdev.showdoc import show_doc

In [ ]:
#export
class Criterion():
    """
    A parent class that inherits all criteria for both classical and learned methods. Criteria can be used as objective or loss functions, as well as evaluation metrics.
    """
    def __init__(
        self,
        name:str, # The name of this criterion which will be monitored in logging.
        supervised:bool, # Whether the criterion is supervised or not.
        differentiable:bool=True, # Whether the criterion is differentiable or not. Only differentiable criteria can be used as loss/objective functions.
        lower_is_better:bool=True, # Whether lower values of the criterion correspond to better scores.
        compute_only_on_design_space:bool=True # Whether the criterion should be evaluated on voxels that have a design space information of -1, i.e., the voxels can be freely optimized. This parameter does not effect all criteria.
    ):
        self._name = name
        self._supervised = supervised
        self._differentiable = differentiable
        self._lower_is_better = lower_is_better
        self.compute_only_on_design_space = compute_only_on_design_space


    @property
    def name(self):
        return self._name


    @property
    def supervised(self):
        return self._supervised


    @property
    def differentiable(self):
        return self._differentiable


    @property
    def lower_is_better(self):
        return self._lower_is_better


    def get_θ_flat(self,
                   solutions:list, # A list of solutions from which the densities are extracted and flattened into one output tensor.
                   binary:bool=False # Whether the densities should be binarized.
                  ):
        """
        Returns a flattened density distribution tensor from the passed solutions. 
        """
        θ = torch.stack([solution.get_θ(binary=binary).flatten() for solution in solutions])
        θ.clamp_(0,1)
        return θ


    def get_design_space_mask(self,
                              solutions:list # A list of solutions from which the design space mask is extracted and combined into a single output tensor.
                             ):
        """
        Returns a flattened design space mask from the passed solutions. If `compute_only_on_design_space=False`, then a ones-vector is returned.
        """
        device = solutions[0].get_θ().device
        if self.compute_only_on_design_space:
            return torch.stack([solution.problem.Ω_design.flatten() == -1 for solution in solutions]).to(device)
        else:
            shape_flat = torch.numel(solutions[0].get_θ())
            return torch.ones(len(solutions), shape_flat, dtype=torch.bool).to(device)


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Only used if `Criterion.supervised=True`.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions. The gt_solutions are only used if `self.criterion.supervised=True`.
        """
        raise NotImplementedError("Must be overridden.")


    def __add__(self, 
                criterion:"dl4to.criteria.Criterion" # A second criterion that should be combined with the current one.
               ):
        """
        The summation of two criteria results in a new combined criterion. Returns a `dl4to.criteria.CombinedCriterion` object.
        """
        combined_criterion = CombinedCriterion(self, criterion)
        return combined_criterion


    def __rmul__(self, 
                 λ:float # The multiplier which the criterion is weighted with.
                ):
        """
        The multiplication of a criterion with a scalar results in a weighted criterion. Returns a `dl4to.criteria.WeightedCriterion` object.
        """
        return WeightedCriterion(self, λ)


    def __mul__(self, 
                 λ:float # The multiplier which the criterion is weighted with.
                ):
        """
        The multiplication of a criterion with a scalar results in a weighted criterion. Returns a `dl4to.criteria.WeightedCriterion` object.
        """
        return self.__rmul__(λ)

In [ ]:
show_doc(Criterion.get_θ_flat)

<h4 id="Criterion.get_θ_flat" class="doc_header"><code>Criterion.get_θ_flat</code><a href="__main__.py#L41" class="source_link" style="float:right">[source]</a></h4>

> <code>Criterion.get_θ_flat</code>(**`solutions`**:`list`, **`binary`**:`bool`=*`False`*)

Returns a flattened density distribution tensor from the passed solutions. 

||Type|Default|Details|
|---|---|---|---|
|**`solutions`**|`list`||A list of solutions from which the densities are extracted and flattened into one output tensor.|
|**`binary`**|`bool`|`False`|Whether the densities should be binarized.|


In [ ]:
show_doc(Criterion.get_design_space_mask)

<h4 id="Criterion.get_design_space_mask" class="doc_header"><code>Criterion.get_design_space_mask</code><a href="__main__.py#L53" class="source_link" style="float:right">[source]</a></h4>

> <code>Criterion.get_design_space_mask</code>(**`solutions`**:`list`)

Returns a flattened design space mask from the passed solutions. If `compute_only_on_design_space=False`, then a ones-vector is returned.

||Type|Default|Details|
|---|---|---|---|
|**`solutions`**|`list`||A list of solutions from which the design space mask is extracted and combined into a single output tensor.|


In [ ]:
show_doc(Criterion.__call__)

<h4 id="Criterion.__call__" class="doc_header"><code>Criterion.__call__</code><a href="__main__.py#L67" class="source_link" style="float:right">[source]</a></h4>

> <code>Criterion.__call__</code>(**`solutions`**:`list`, **`gt_solutions`**:`list`=*`None`*, **`binary`**:`bool`=*`False`*)

Calculates the output of the criterion for all solutions. The gt_solutions are only used if `self.criterion.supervised=True`.

||Type|Default|Details|
|---|---|---|---|
|**`solutions`**|`list`||The solutions that should be evaluated with the criterion.|
|**`gt_solutions`**|`list`|`None`|Ground truth solutions that are compared element-wise with the `solutions`. Only used if [`Criterion.supervised=True`](/dl4tocriteria.html#Criterion.supervised=True).|
|**`binary`**|`bool`|`False`|Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.|


In [ ]:
show_doc(Criterion.__add__)

<h4 id="Criterion.__add__" class="doc_header"><code>Criterion.__add__</code><a href="__main__.py#L78" class="source_link" style="float:right">[source]</a></h4>

> <code>Criterion.__add__</code>(**`criterion`**:`dl4to.criteria.Criterion`)

The summation of two criteria results in a new combined criterion. Returns a `dl4to.criteria.CombinedCriterion` object.

||Type|Default|Details|
|---|---|---|---|
|**`criterion`**|`dl4to.criteria.Criterion`||A second criterion that should be combined with the current one.|


In [ ]:
show_doc(Criterion.__mul__)

<h4 id="Criterion.__mul__" class="doc_header"><code>Criterion.__mul__</code><a href="__main__.py#L97" class="source_link" style="float:right">[source]</a></h4>

> <code>Criterion.__mul__</code>(**`λ`**:`float`)

The multiplication of a criterion with a scalar results in a weighted criterion. Returns a `dl4to.criteria.WeightedCriterion` object.

||Type|Default|Details|
|---|---|---|---|
|**`λ`**|`float`||The multiplier which the criterion is weighted with.|


In [ ]:
#export
class WeightedCriterion(Criterion):
    """
    A class that represents a criterion that has a weight factor in front of it. This is especially useful for constrained optimization or regularization.
    Note that the unweighted criterion can be accessed via `self.criterion`.
    """
    def __init__(self, 
                 criterion:"dl4to.criteria.Criterion", # The criterion object that is being weighted.
                 λ:float # The weighting factor for the criterion.
                ):
        self.criterion = criterion
        self.λ = λ
        lower_is_better = (not self.criterion.lower_is_better) if self.λ < 0 else (self.criterion.lower_is_better)

        super().__init__(
            name=f'{self.λ}_{self.criterion.name}',
            supervised=self.criterion.supervised,
            differentiable=self.criterion.differentiable,
            lower_is_better=lower_is_better,
            compute_only_on_design_space=self.criterion.compute_only_on_design_space
        )


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Only used if `Criterion.supervised=True`.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions. The gt_solutions are only used if `self.criterion.supervised=True`.
        """
        return self.λ * self.criterion(solutions, gt_solutions, binary)

In [ ]:
#export
class CombinedCriterion(Criterion):
    """
    A class that represents the combination of two criteria by a plus sign "+" between them. Both individual criteria can be accessed via `self.criterion1` and `self.criterion2`.
    """
    def __init__(self, 
                 criterion1:"dl4to.criteria.Criterion", # The first criterion of the summation.
                 criterion2:"dl4to.criteria.Criterion" # The second criterion of the summation.
                ):
        self.criterion1 = criterion1
        self.criterion2 = criterion2
        name = f"{self.criterion1.name}_plus_{self.criterion2.name}"
        supervised=self.criterion1.supervised or self.criterion2.supervised
        differentiable = self.criterion1.differentiable and self.criterion2.differentiable
        if self.criterion1.lower_is_better != self.criterion2.lower_is_better:
            raise AttributeError("Cannot combine two criteria with different values in `lower_is_better`.")
        lower_is_better = self.criterion1.lower_is_better
        compute_only_on_design_space = self.criterion1.compute_only_on_design_space and self.criterion2.compute_only_on_design_space
        super().__init__(
            name=name,
            supervised=supervised,
            differentiable=differentiable,
            lower_is_better=lower_is_better,
            compute_only_on_design_space=compute_only_on_design_space
        )


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Only used if `Criterion.supervised=True`.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions. The gt_solutions are only used if `self.criterion.supervised=True`.
        """
        criterion1_vals = self.criterion1(solutions, gt_solutions, binary)
        criterion2_vals = self.criterion2(solutions, gt_solutions, binary)
        if criterion1_vals.device != criterion2_vals.device:
            if criterion1_vals.device == torch.device('cpu'):
                criterion1_vals = criterion1_vals.to(criterion2_vals.device)
            if criterion2_vals.device == torch.device('cpu'):
                criterion2_vals = criterion2_vals.to(criterion1_vals.device)
        return criterion1_vals + criterion2_vals